In [2]:
import plotly.express as px
import polars as pl
import pandas as pd
import bar_chart_race as bcr

In [7]:
df = pl.read_csv("./output.csv")

# exact matches only
drugs = df.filter(pl.col("Similarity Score") == 1.0)

speeches = pl.read_csv("./speech_table.csv")

df = pd.merge(
    drugs.to_pandas(),
    speeches.to_pandas(),
    left_on='Row ID',
    right_on='index',
)
df.drop(columns=['Source Column', 'Similarity Score'], inplace=True)

del speeches, drugs
df['date'] = pd.to_datetime(df['date'])

groups = df.groupby([pd.Grouper(key='date', freq='M'), 'Search Term']).size().reset_index().rename(columns={0: 'count'})
groups.head()

,date,Search Term,count
0,1847-03-31,HEMP,1
1,1855-03-31,HEMP,1
2,1855-12-31,HEMP,1
3,1864-02-29,HEMP,1
4,1867-12-31,HEMP,1


In [12]:
df = groups.pivot(index='date', columns='Search Term', values='count').fillna(0).cumsum()
df

Search Term,ACETYL,ALCOHOL,AMPHETAMINE,ASPIRIN,BARBITURATES,BUPRENORPHINE,CANNABIS,CARFENTANIL,CBD,COCAINE,...,METHYLENEDIOXYMETHAMPHETAMINE,MORPHINE,NALOXONE,NARCAN,NICOTINE,OPIATE,OPIOID,OXYCODONE,PCP,PHENCYCLIDINE
date,,,,,,,,,,,,,,,,,,,,,
1847-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855-03-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1855-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1864-02-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1867-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31,1.0,1053.0,38.0,32.0,26.0,16.0,26.0,3.0,5.0,609.0,...,6.0,37.0,43.0,11.0,61.0,57.0,547.0,9.0,18.0,8.0
2022-11-30,1.0,1053.0,38.0,34.0,26.0,16.0,26.0,3.0,6.0,609.0,...,6.0,37.0,43.0,11.0,61.0,57.0,550.0,9.0,18.0,8.0
2022-12-31,1.0,1054.0,38.0,34.0,26.0,16.0,26.0,3.0,7.0,611.0,...,6.0,37.0,43.0,11.0,61.0,57.0,554.0,9.0,18.0,8.0


In [34]:
bcr.bar_chart_race(
    df=df,
    # df.iloc[200:400, :12], 
    filename="animated_presidency_drugs_bars.mp4",
    # filename=None,
    n_bars=10,
    period_length=50,
    period_fmt="%B %Y",
    period_summary_func=lambda v, _: {
        'x': .98,
        'y': .05, 
        's': f'Total mentions: {v.sum():,.0f}', 
        'ha': 'right', 
        'size': 10,
    },
    title="Presidency Drugs",
)

/opt/homebrew/Caskroom/miniconda/base/envs/pres/lib/python3.11/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/homebrew/Caskroom/miniconda/base/envs/pres/lib/python3.11/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
/opt/homebrew/Caskroom/miniconda/base/envs/pres/lib/python3.11/site-packages/bar_chart_race/_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "
